In [1]:
using Distributions
using LinearAlgebra
using Plots
using Sobol
using Random
using Profile
using BenchmarkTools

In [4]:
include("../testfns.jl")
include("../rollout.jl")

distributed_rollout_solver (generic function with 1 method)

In [5]:
function generate_initial_guesses(N::Int, lbs::Vector{T}, ubs::Vector{T},) where T <: Number
    ϵ = 1e-6
    seq = SobolSeq(lbs, ubs)
    initial_guesses = reduce(hcat, next!(seq) for i = 1:N)
    initial_guesses = hcat(initial_guesses, lbs .+ ϵ)
    initial_guesses = hcat(initial_guesses, ubs .- ϵ)

    return initial_guesses
end

generate_initial_guesses (generic function with 1 method)

In [25]:
testfn_payloads = Dict(
    "gramacylee" => (name="gramacylee", fn=TestGramacyLee, args=()),
    "rastrigin" => (name="rastrigin", fn=TestRastrigin, args=(1)),
    "ackley1d" => (name="ackley1d", fn=TestAckley, args=(1)),
    "ackley2d" => (name="ackley2d", fn=TestAckley, args=(2)),
    "ackley3d" => (name="ackley3d", fn=TestAckley, args=(3)),
    "ackley4d" => (name="ackley4d", fn=TestAckley, args=(4)),
    "ackley10d" => (name="ackley10d", fn=TestAckley, args=(2)),
    "rosenbrock" => (name="rosenbrock", fn=TestRosenbrock, args=()),
    "sixhump" => (name="sixhump", fn=TestSixHump, args=()),
    "braninhoo" => (name="braninhoo", fn=TestBraninHoo, args=()),
    "hartmann3d" => (name="hartmann3d", fn=TestHartmann3D, args=()),
    "goldsteinprice" => (name="goldsteinprice", fn=TestGoldsteinPrice, args=()),
    "beale" => (name="beale", fn=TestBeale, args=()),
    "easom" => (name="easom", fn=TestEasom, args=()),
    "styblinskitang1d" => (name="styblinskitang1d", fn=TestStyblinskiTang, args=(1)),
    "styblinskitang2d" => (name="styblinskitang2d", fn=TestStyblinskiTang, args=(2)),
    "styblinskitang3d" => (name="styblinskitang3d", fn=TestStyblinskiTang, args=(3)),
    "styblinskitang4d" => (name="styblinskitang4d", fn=TestStyblinskiTang, args=(4)),
    "styblinskitang10d" => (name="styblinskitang10d", fn=TestStyblinskiTang, args=(10)),
    "bukinn6" => (name="bukinn6", fn=TestBukinN6, args=()),
    "crossintray" => (name="crossintray", fn=TestCrossInTray, args=()),
    "eggholder" => (name="eggholder", fn=TestEggHolder, args=()),
    "holdertable" => (name="holdertable", fn=TestHolderTable, args=()),
    "schwefel1d" => (name="schwefel1d", fn=TestSchwefel, args=(1)),
    "schwefel2d" => (name="schwefel2d", fn=TestSchwefel, args=(2)),
    "schwefel3d" => (name="schwefel3d", fn=TestSchwefel, args=(3)),
    "schwefel4d" => (name="schwefel4d", fn=TestSchwefel, args=(4)),
    "schwefel10d" => (name="schwefel10d", fn=TestSchwefel, args=(10)),
    "levyn13" => (name="levyn13", fn=TestLevyN13, args=()),
    "trid1d" => (name="trid1d", fn=TestTrid, args=(1)),
    "trid2d" => (name="trid2d", fn=TestTrid, args=(2)),
    "trid3d" => (name="trid3d", fn=TestTrid, args=(3)),
    "trid4d" => (name="trid4d", fn=TestTrid, args=(4)),
    "trid10d" => (name="trid10d", fn=TestTrid, args=(10)),
    "mccormick" => (name="mccormick", fn=TestMccormick, args=()),
    "hartmann6d" => (name="hartmann6d", fn=TestHartmann6D, args=()),
    "hartmann4d" => (name="hartmann4d", fn=TestHartmann4D, args=()),
    "hartmann3d" => (name="hartmann3d", fn=TestHartmann3D, args=()),
)

# Paramaters
MC_SAMPLES = 1
NUMBER_OF_STARTS = 8
NUMBER_OF_TRIALS = 1
trial = 1
HORIZON = 1
BUDGET = 1
BATCH_SIZE = 16

# Gaussian process hyperparameters
θ, σn2 = [1.], 1e-6
ψ = kernel_matern52(θ)

# Build the test function object
payload = testfn_payloads["hartmann3d"]
# println("Running experiment for $(payload.name).")
testfn = payload.fn(payload.args...)
lbs, ubs = testfn.bounds[:,1], testfn.bounds[:,2]

# Generate low discrepancy sequence
lds_rns = gen_low_discrepancy_sequence(MC_SAMPLES, testfn.dim, HORIZON + 1)

# Allocate initial guesses for optimizer
initial_guesses = generate_initial_guesses(NUMBER_OF_STARTS, lbs, ubs)

# Allocate all initial samples
initial_samples = randsample(NUMBER_OF_TRIALS, testfn.dim, lbs, ubs)

# Allocate space for GAPS
rollout_gaps = zeros(BUDGET + 1)

# Initialize the trajectory parameters
tp = TrajectoryParameters(
    initial_samples[:, 1], # Will be overriden later
    HORIZON,
    MC_SAMPLES,
    lds_rns,
    lbs,
    ubs,
)

# Initialize batch of points to evaluate the rollout acquisition function
batch = generate_batch(BATCH_SIZE, lbs=tp.lbs, ubs=tp.ubs)

Xinit = initial_samples[:, trial:trial]
yinit = testfn.f.(eachcol(Xinit))
sur = fit_surrogate(ψ, Xinit, yinit; σn2=σn2);
fsur = fit_fsurrogate(sur, 0);

In [157]:
function ei_solve(s::FantasyRBFsurrogate, lbs::Vector{Float64}, ubs::Vector{Float64}
    , xstart::Vector{Float64})
    fun(x) = -s(x).EI
    function fun_grad!(g, x)
        EIx = -s(x).∇EI
        for i in eachindex(EIx)
            g[i] = EIx[i]
        end
    end
    function fun_hess!(h, x)
        HEIx = -s(x).HEI
        for row in 1:size(HEIx, 1)
            for col in 1:size(HEIx, 2)
                h[row, col] = HEIx[row, col]
            end
        end
    end

    df = TwiceDifferentiable(fun, fun_grad!, fun_hess!, xstart)
    dfc = TwiceDifferentiableConstraints(lbs, ubs)
    res = optimize(
        df, dfc, xstart, IPNewton(),
        Optim.Options(x_tol=1e-3, f_tol=1e-3)
    )

    # return Optim.minimizer(res), res, data
    return Optim.minimizer(res), res
end

function multistart_ei_solve(s::FantasyRBFsurrogate, lbs::Vector{Float64},
    ubs::Vector{Float64}, xstarts::Matrix{Float64})::Vector{Float64}
    candidates = []
    
    for i in 1:size(xstarts, 2)
        xi = xstarts[:,i]
        try
            minimizer, res = ei_solve(s, lbs, ubs, xi)
            push!(candidates, (minimizer, minimum(res)))
        catch e
            println(e)
        end
    end
    
    candidates = filter(pair -> !any(isnan.(pair[1])), candidates)
    mini, j_mini = findmin(pair -> pair[2], candidates)
    minimizer = candidates[j_mini][1]

    return minimizer
end

multistart_ei_solve (generic function with 1 method)

In [158]:
@timed multistart_ei_solve(fsur, tp.lbs, tp.ubs, initial_guesses)

(value = [0.07283255961961017, 0.9977083367122132, 0.09409060692233179], time = 0.094896808, bytes = 5084318, gctime = 0.019064252, gcstats = Base.GC_Diff(5084318, 0, 0, 76537, 52, 50, 19064252, 1, 0))

In [154]:
@timed multistart_ei_solve(fsur, tp.lbs, tp.ubs, initial_guesses)

(value = [1.6374213302210285e-9, 0.999999999030225, 1.0901343782409662e-9], time = 0.100165638, bytes = 22675376, gctime = 0.0, gcstats = Base.GC_Diff(22675376, 0, 0, 347582, 0, 0, 0, 0, 0))

In [159]:
using SharedArrays

In [169]:
@allocations SharedMatrix{Float64}(500, 16)

101

In [168]:
@allocations zeros(500, 16)

2